<a href="https://colab.research.google.com/github/osmarbraz/exemplos_python/blob/master/ExemplosSpacyDBPedia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exemplos de Reconhecimento Entidades com sPacyDBPedia

https://pypi.org/project/spacy-dbpedia-spotlight/

-----------------------------------------
**Guia Colab Iniciante:**

https://medium.com/machina-sapiens/google-colab-guia-do-iniciante-334d70aad531

**Biblioteca:**
https://colab.research.google.com/github/jakevdp/PythonDataScienceHandbook/blob/master/notebooks/01.01-Help-And-Documentation.ipynb

**Características :**

https://colab.research.google.com/notebooks/basic_features_overview.ipynb

## Tratamento de logs

In [1]:
# Import das bibliotecas.
import logging # Biblioteca de logging

# Formatando a mensagem de logging
logging.basicConfig(format="%(asctime)s : %(levelname)s : %(message)s", level=logging.INFO)

## Instalação do spaCy

https://spacy.io/

Modelos do spaCy para português:
https://spacy.io/models/pt

In [2]:
# Instala o spacy
!pip install -U pip setuptools wheel

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
# Instala uma versão específica
!pip install -U spacy==3.2.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Download arquivo modelo

https://spacy.io/models/pt

Realiza o download e um arquivo

In [4]:
# Import das bibliotecas.
import requests # Biblioteca de download
from tqdm.notebook import tqdm as tqdm_notebook # Biblioteca para barra de progresso
import os # Biblioteca para manipular arquivos

def downloadArquivo(url_arquivo, nome_arquivo_destino):
    """    
      Realiza o download de um arquivo de uma url em salva em nome_arquivo_destino.
    
      Parâmetros:
        `url_arquivo` - URL do arquivo a ser feito download.      
        `nome_arquivo_destino` - Nome do arquivo a ser salvo.      
    """
    
    # Verifica se existe o diretório base
    DIRETORIO_COHEBERT = '/content'
    
    # Realiza o download de um arquivo em uma url
    data = requests.get(url_arquivo, stream=True)
    
    # Verifica se o arquivo existe
    if data.status_code != 200:
        logging.info("Exceção ao tentar realizar download {}. Response {}.".format(url_arquivo, data.status_code))
        data.raise_for_status()
        return

    # Recupera o nome do arquivo a ser realizado o download    
    nome_arquivo = nome_arquivo_destino.split("/")[-1]  

    # Define o nome e caminho do arquivo temporário    
    nome_arquivo_temporario = DIRETORIO_COHEBERT + "/" + nome_arquivo + "_part"
    
    logging.info("Download do arquivo: {}.".format(nome_arquivo_destino))
    
    # Baixa o arquivo
    with open(nome_arquivo_temporario, "wb") as arquivo_binario:        
        tamanho_conteudo = data.headers.get("Content-Length")        
        total = int(tamanho_conteudo) if tamanho_conteudo is not None else None
        # Barra de progresso de download
        progresso_bar = tqdm_notebook(unit="B", total=total, unit_scale=True)                
        # Atualiza a barra de progresso
        for chunk in data.iter_content(chunk_size=1024):        
            if chunk:                
                progresso_bar.update(len(chunk))
                arquivo_binario.write(chunk)
    
    # Renomeia o arquivo temporário para o arquivo definitivo
    os.rename(nome_arquivo_temporario, nome_arquivo_destino)
    
    # Fecha a barra de progresso.
    progresso_bar.close()

### Função download modelo spaCy

In [5]:
def downloadSpacy(modelo_spacy,versao_modelo_spacy):
    """
      Realiza o download do arquivo do modelo para o diretório corrente.
    
      Parâmetros:
        `modelo_spacy` - Modelo do space.
        `versao_modelo_spacy` - Versão do modelo do space.       
    """
    # Verifica se existe o diretório base
    DIRETORIO_COHEBERT = '/content'
        
    # Nome arquivo spacy
    ARQUIVO_MODELO_SPACY = modelo_spacy
    # Versão spaCy
    VERSAO_SPACY = "-" + versao_modelo_spacy
    # Nome arquivo compactado
    NOME_ARQUIVO_MODELO_COMPACTADO = ARQUIVO_MODELO_SPACY + VERSAO_SPACY + ".tar.gz"
    
    # Url do arquivo
    URL_ARQUIVO_MODELO_COMPACTADO = "https://github.com/explosion/spacy-models/releases/download/" + ARQUIVO_MODELO_SPACY + VERSAO_SPACY + "/" + NOME_ARQUIVO_MODELO_COMPACTADO

    # Realiza o download do arquivo do modelo
    logging.info("Download do arquivo do modelo do spaCy.")
    downloadArquivo(URL_ARQUIVO_MODELO_COMPACTADO, DIRETORIO_COHEBERT + "/" + NOME_ARQUIVO_MODELO_COMPACTADO)

## Descompacta o arquivo do modelo

### Função descompacta modelo spaCy

In [6]:
# Import das bibliotecas.
import tarfile # Biblioteca de descompactação

def descompactaSpacy(modelo_spacy, versao_modelo_spacy):
    """
      Descompacta o arquivo do modelo.
    
      Parâmetros:
        `modelo_spacy` - Modelo do space.
        `versao_modelo_spacy` - Versão do modelo do space.
    """
    
    # Verifica se existe o diretório base do cohebert e retorna o nome do diretório
    DIRETORIO_COHEBERT = '/content'
    
    # Nome arquivo spacy
    ARQUIVO_MODELO_SPACY = modelo_spacy
    # Versão spaCy
    VERSAO_SPACY = "-" + versao_modelo_spacy
    
    # Nome do arquivo a ser descompactado
    NOME_ARQUIVO_MODELO_COMPACTADO = DIRETORIO_COHEBERT + "/" + ARQUIVO_MODELO_SPACY + VERSAO_SPACY + ".tar.gz"
    
    logging.info("Descompactando o arquivo do modelo do spaCy.")
    arquivo_tar = tarfile.open(NOME_ARQUIVO_MODELO_COMPACTADO, "r:gz")    
    arquivo_tar.extractall(DIRETORIO_COHEBERT)    
    arquivo_tar.close()
    
    # Apaga o arquivo compactado
    if os.path.isfile(NOME_ARQUIVO_MODELO_COMPACTADO):        
        os.remove(NOME_ARQUIVO_MODELO_COMPACTADO)

## Carrega o modelo

### Função carrega modelo spaCy

In [7]:
# Import das bibliotecas.
import spacy # Biblioteca do spaCy

def carregaSpacy(modelo_spacy, versao_modelo_spacy):
    """
    Realiza o carregamento do Spacy.
    
    Parâmetros:
        `modelo_spacy` - Modelo do space.
        `versao_modelo_spacy` - Versão do modelo do space.           
    """
    
    # Verifica se existe o diretório base
    DIRETORIO_COHEBERT = "/content"
                  
    # Nome arquivo spacy
    ARQUIVO_MODELO_SPACY = modelo_spacy
    # Versão spaCy
    VERSAO_SPACY = "-" + versao_modelo_spacy
    # Caminho raoz do modelo do spaCy
    DIRETORIO_MODELO_SPACY =  DIRETORIO_COHEBERT + "/" + ARQUIVO_MODELO_SPACY + VERSAO_SPACY

    # Verifica se o diretório existe
    if os.path.exists(DIRETORIO_MODELO_SPACY) == False:
        # Realiza o download do arquivo modelo do spaCy
        downloadSpacy(modelo_spacy, versao_modelo_spacy)
        # Descompacta o spaCy
        descompactaSpacy(modelo_spacy, versao_modelo_spacy)

    # Diretório completo do spaCy
    DIRETORIO_MODELO_SPACY = DIRETORIO_COHEBERT + "/" + ARQUIVO_MODELO_SPACY + VERSAO_SPACY + "/" + ARQUIVO_MODELO_SPACY + "/" + ARQUIVO_MODELO_SPACY + VERSAO_SPACY + "/"

    # Carrega o spaCy. Necessário somente "tagger" para encontrar os substantivos
    nlp = spacy.load(DIRETORIO_MODELO_SPACY)
    logging.info("spaCy carregado.")

    # Retorna o spacy carregado
    return nlp 

### Carrega o modelo spaCy


In [8]:
modelo_spacy = "pt_core_news_lg"
#modelo_spacy = "pt_core_news_md"
#modelo_spacy = "pt_core_news_sm"
versao_modelo_spacy = "3.2.0"

# Carrega o modelo spaCy
nlp1 = carregaSpacy(modelo_spacy, versao_modelo_spacy)

## SpaCy DBpedia Spotlight

https://pypi.org/project/spacy-dbpedia-spotlight/

## Instalação

https://github.com/aghie/pybabelfy

In [9]:
!pip install spacy-dbpedia-spotlight

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Exemplos

### Adicionando a um modelo spaCy já carregado

In [10]:
import spacy


nlp1.add_pipe('dbpedia_spotlight')

print(nlp1.pipe_names)

['tok2vec', 'morphologizer', 'parser', 'attribute_ruler', 'lemmatizer', 'ner', 'dbpedia_spotlight']


In [11]:
texto = "Como empilhar e desempilhar elementos em uma estrutura de dados pilha ?"

doc = nlp1(texto)

for ent in doc.ents:
  print(ent.text, ent.kb_id_, ent._.dbpedia_raw_result)


estrutura de dados http://pt.dbpedia.org/resource/Estrutura_de_dados {'@URI': 'http://pt.dbpedia.org/resource/Estrutura_de_dados', '@support': '194', '@types': '', '@surfaceForm': 'estrutura de dados', '@offset': '45', '@similarityScore': '0.9999999992349871', '@percentageOfSecondRank': '7.650195037466937E-10'}


### Carregando um modelo em branco

In [12]:
import spacy_dbpedia_spotlight

nlp = spacy_dbpedia_spotlight.create('pt')

texto = "Como empilhar e desempilhar elementos em uma estrutura de dados pilha ?"

doc = nlp(texto)

for ent in doc.ents:
  print(ent.text, ent.kb_id_, ent._.dbpedia_raw_result)

estrutura de dados http://pt.dbpedia.org/resource/Estrutura_de_dados {'@URI': 'http://pt.dbpedia.org/resource/Estrutura_de_dados', '@support': '194', '@types': '', '@surfaceForm': 'estrutura de dados', '@offset': '45', '@similarityScore': '0.9999999992349871', '@percentageOfSecondRank': '7.650195037466937E-10'}


In [13]:
# Somente organizações
nlp.get_pipe('dbpedia_spotlight').types = 'DBpedia:Organisation'

# Recria o documento
doc = nlp(texto)

# Mostra a saída
print([(ent.text, ent.kb_id_, ent._.dbpedia_raw_result['@similarityScore']) for ent in doc.ents])

[]


In [14]:
# Agora tudo junto
nlp.get_pipe('dbpedia_spotlight').types = None

# Recria o documento
doc = nlp(texto)

# Mostar a saída
print([(ent.text, ent.kb_id_, ent._.dbpedia_raw_result['@similarityScore']) for ent in doc.ents])

[('estrutura de dados', 'http://pt.dbpedia.org/resource/Estrutura_de_dados', '0.9999999992349871')]
